In [1]:
from scipy.stats import pearsonr
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WhitespaceTokenizer
from nltk.metrics import jaccard_distance
import csv
from nltk import ne_chunk,pos_tag,word_tokenize
from nltk.chunk import tree2conllstr
from nltk.tree import Tree
from nltk.parse import CoreNLPParser

In [2]:
parser = CoreNLPParser(url='http://localhost:9000', tagtype='ner')
# java -mx4g -cp C:\stanford-corenlp-full-2018-10-05\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

In [41]:
trial_path = 'data/trial/STS.input.txt'
trial_gs_path = 'data/trial/STS.gs.txt'
trial_df = pd.read_csv(trial_path, sep='\t', lineterminator='\n', names=['sentence0','sentence1'], header=None, quoting=csv.QUOTE_NONE)
trial_gs = pd.read_csv(trial_gs_path, sep='\t', lineterminator='\n', names=['labels'], header=None, quoting=csv.QUOTE_NONE)

def na_parser(tagged):
    parsed     = []
    last_tag   = None
    start_index = 0
    for index, node in enumerate(tagged):
        tag = node[1]
        if (tag == 'O' or tag != last_tag) and (start_index != index):
            token = ' '.join([pair[0].lower() for pair in tagged[start_index:index]])
            if token.isalnum():
                parsed.append(token)
            last_tag = tag
            start_index = index
    return parsed


def preprocessing(data):
    # To see the effect of the desambiguation we do a preprocess only with it. Generaly we will use a more complete 
    # preprocessing function. 
    data = data.fillna('')
    first = lambda x: [a[0] for a in x]
    for column in data.columns:
        # words to lower
        #data[column] = data[column].str.lower()
        # desambiguate 
        data[column] = data[column].apply(word_tokenize)
        data[column] = data[column].apply(parser.tag)
        data[column] = data[column].apply(na_parser)
        data[column].apply(print)
    return data

trial_df = preprocessing(trial_df)
trial_df

['the', 'bird', 'is', 'bathing', 'in', 'the', 'sink']
['in', 'the', 'troops', 'attempted', 'to', 'invade', 'kabul']
['john', 'said', 'he', 'is', 'considered', 'a', 'witness', 'but', 'not', 'a', 'suspect']
['they', 'flew', 'out', 'of', 'the', 'nest', 'in', 'groups']
['the', 'woman', 'is', 'playing', 'the', 'violin']
['john', 'went', 'horse', 'back', 'riding', 'at', 'dawn', 'with', 'a', 'whole', 'group', 'of', 'friends']
['birdie', 'is', 'washing', 'itself', 'in', 'the', 'water', 'basin']
['the', 'us', 'army', 'invaded', 'kabul', 'on']
['he', 'is', 'not', 'a', 'suspect', 'anymore', 'john', 'said']
['they', 'flew', 'into', 'the', 'nest', 'together']
['the', 'young', 'lady', 'enjoys', 'listening', 'to', 'the', 'guitar']
['sunrise', 'at', 'dawn', 'is', 'a', 'magnificent', 'view', 'to', 'take', 'in', 'if', 'you', 'wake', 'up', 'early', 'enough', 'for', 'it']


,sentence0,sentence1
id1,"[the, bird, is, bathing, in, the, sink]","[birdie, is, washing, itself, in, the, water, ..."
id2,"[in, the, troops, attempted, to, invade, kabul]","[the, us, army, invaded, kabul, on]"
id3,"[john, said, he, is, considered, a, witness, b...","[he, is, not, a, suspect, anymore, john, said]"
id4,"[they, flew, out, of, the, nest, in, groups]","[they, flew, into, the, nest, together]"
id5,"[the, woman, is, playing, the, violin]","[the, young, lady, enjoys, listening, to, the,..."
id6,"[john, went, horse, back, riding, at, dawn, wi...","[sunrise, at, dawn, is, a, magnificent, view, ..."


In [43]:
def lexical_simmilarity(df):
    guess = pd.DataFrame()
    for i in df.index:
        guess.loc[i,'labels'] = 1. - jaccard_distance(set(df.loc[i,'sentence0']), set(df.loc[i,'sentence1']))
    return guess

In [44]:
guess_lex = lexical_simmilarity(trial_df)
print(guess_lex)

       labels
id1  0.272727
id2  0.181818
id3  0.636364
id4  0.400000
id5  0.090909
id6  0.107143


In [45]:
print(pearsonr(trial_gs['labels'], guess_lex['labels'])[0])

-0.34327956222578704


The results are even worse than the first ones and i don't know why. 